# Chapter 29: Progressive Web Applications

Transform your Next.js application into an installable, offline-capable Progressive Web App (PWA) that delivers native-like experiences while maintaining the web's reach and flexibility. Modern PWAs leverage service workers for offline functionality, background sync for deferred actions, and web app manifests for seamless installation across devices.

By the end of this chapter, you'll master configuring service workers with next-pwa, implementing offline fallback strategies, managing background synchronization, handling installation prompts, and optimizing your PWA for performance and reliability.

## 29.1 PWA Configuration with next-pwa

Configure your Next.js application to generate service workers and web app manifests automatically while maintaining full control over caching strategies.

### Basic PWA Setup

```typescript
// next.config.js
const withPWA = require('next-pwa')({
  dest: 'public',
  register: true,
  skipWaiting: true,
  disable: process.env.NODE_ENV === 'development',
  buildExcludes: [/middleware-manifest\.json$/],
  runtimeCaching: [
    {
      urlPattern: /^https:\/\/fonts\.(?:gstatic)\.com\/.*/i,
      handler: 'CacheFirst',
      options: {
        cacheName: 'google-fonts-webfonts',
        expiration: {
          maxEntries: 4,
          maxAgeSeconds: 365 * 24 * 60 * 60, // 1 year
        },
      },
    },
    {
      urlPattern: /\.(?:eot|otf|ttc|ttf|woff|woff2|font.css)$/i,
      handler: 'StaleWhileRevalidate',
      options: {
        cacheName: 'static-font-assets',
        expiration: {
          maxEntries: 4,
          maxAgeSeconds: 7 * 24 * 60 * 60, // 7 days
        },
      },
    },
    {
      urlPattern: /\/_next\/image\?url=.+$/i,
      handler: 'StaleWhileRevalidate',
      options: {
        cacheName: 'next-image',
        expiration: {
          maxEntries: 64,
          maxAgeSeconds: 24 * 60 * 60, // 24 hours
        },
      },
    },
    {
      urlPattern: /\/_next\/static\/.+/i,
      handler: 'CacheFirst',
      options: {
        cacheName: 'next-static',
        expiration: {
          maxEntries: 200,
          maxAgeSeconds: 365 * 24 * 60 * 60, // 1 year
        },
      },
    },
    {
      urlPattern: /\/api\/.*$/i,
      handler: 'NetworkFirst',
      options: {
        cacheName: 'apis',
        expiration: {
          maxEntries: 16,
          maxAgeSeconds: 24 * 60 * 60, // 24 hours
        },
        networkTimeoutSeconds: 10,
      },
    },
    {
      urlPattern: /^\/(?!api\/).*$/i, // All pages except API
      handler: 'NetworkFirst',
      options: {
        cacheName: 'pages',
        expiration: {
          maxEntries: 32,
          maxAgeSeconds: 24 * 60 * 60, // 24 hours
        },
      },
    },
  ],
});

module.exports = withPWA({
  // Your existing Next.js config
  reactStrictMode: true,
});
```

### Web App Manifest Configuration

Create a comprehensive manifest to control how your app appears when installed:

```json
// public/manifest.json
{
  "name": "Next.js Mastery Guide",
  "short_name": "MasteryGuide",
  "description": "Interactive learning platform for Next.js development",
  "start_url": "/",
  "display": "standalone",
  "background_color": "#ffffff",
  "theme_color": "#000000",
  "orientation": "portrait-primary",
  "scope": "/",
  "id": "/",
  "icons": [
    {
      "src": "/icons/icon-72x72.png",
      "sizes": "72x72",
      "type": "image/png",
      "purpose": "maskable any"
    },
    {
      "src": "/icons/icon-96x96.png",
      "sizes": "96x96",
      "type": "image/png",
      "purpose": "maskable any"
    },
    {
      "src": "/icons/icon-128x128.png",
      "sizes": "128x128",
      "type": "image/png",
      "purpose": "maskable any"
    },
    {
      "src": "/icons/icon-144x144.png",
      "sizes": "144x144",
      "type": "image/png",
      "purpose": "maskable any"
    },
    {
      "src": "/icons/icon-152x152.png",
      "sizes": "152x152",
      "type": "image/png",
      "purpose": "maskable any"
    },
    {
      "src": "/icons/icon-192x192.png",
      "sizes": "192x192",
      "type": "image/png",
      "purpose": "maskable any"
    },
    {
      "src": "/icons/icon-384x384.png",
      "sizes": "384x384",
      "type": "image/png",
      "purpose": "maskable any"
    },
    {
      "src": "/icons/icon-512x512.png",
      "sizes": "512x512",
      "type": "image/png",
      "purpose": "maskable any"
    }
  ],
  "shortcuts": [
    {
      "name": "Dashboard",
      "short_name": "Dashboard",
      "description": "View your learning progress",
      "url": "/dashboard",
      "icons": [{ "src": "/icons/dashboard.png", "sizes": "96x96" }]
    },
    {
      "name": "Courses",
      "short_name": "Courses",
      "description": "Browse available courses",
      "url": "/courses",
      "icons": [{ "src": "/icons/courses.png", "sizes": "96x96" }]
    }
  ],
  "categories": ["education", "productivity"],
  "screenshots": [
    {
      "src": "/screenshots/dashboard-wide.png",
      "sizes": "1280x720",
      "type": "image/png",
      "form_factor": "wide",
      "label": "Dashboard view"
    },
    {
      "src": "/screenshots/mobile-home.png",
      "sizes": "750x1334",
      "type": "image/png",
      "form_factor": "narrow",
      "label": "Home screen on mobile"
    }
  ],
  "related_applications": [],
  "prefer_related_applications": false
}
```

## 29.2 Advanced Service Worker Strategies

Implement custom service worker logic for complex offline scenarios beyond the basic next-pwa configuration.

### Custom Service Worker Implementation

```typescript
// public/service-worker.js
import { skipWaiting, clientsClaim } from 'workbox-core';
import { precacheAndRoute, cleanupOutdatedCaches } from 'workbox-precaching';
import { registerRoute, setCatchHandler } from 'workbox-routing';
import { StaleWhileRevalidate, CacheFirst, NetworkFirst } from 'workbox-strategies';
import { ExpirationPlugin } from 'workbox-expiration';
import { BackgroundSyncPlugin } from 'workbox-background-sync';

skipWaiting();
clientsClaim();

// Precache static assets generated by Next.js
precacheAndRoute(self.__WB_MANIFEST || []);
cleanupOutdatedCaches();

// API routes: Network first with background sync
registerRoute(
  ({ url }) => url.pathname.startsWith('/api/'),
  new NetworkFirst({
    cacheName: 'api-cache',
    plugins: [
      new BackgroundSyncPlugin('api-queue', {
        maxRetentionTime: 24 * 60, // Retry for max of 24 hours
      }),
      new ExpirationPlugin({
        maxEntries: 100,
        maxAgeSeconds: 7 * 24 * 60 * 60, // 7 days
      }),
    ],
  })
);

// Images: Cache first with expiration
registerRoute(
  ({ request }) => request.destination === 'image',
  new CacheFirst({
    cacheName: 'image-cache',
    plugins: [
      new ExpirationPlugin({
        maxEntries: 200,
        maxAgeSeconds: 30 * 24 * 60 * 60, // 30 days
        purgeOnQuotaError: true,
      }),
    ],
  })
);

// Fonts: Cache first, long term
registerRoute(
  ({ url }) => url.origin.includes('fonts.gstatic.com') || url.pathname.includes('.woff2'),
  new CacheFirst({
    cacheName: 'font-cache',
    plugins: [
      new ExpirationPlugin({
        maxEntries: 30,
        maxAgeSeconds: 365 * 24 * 60 * 60, // 1 year
      }),
    ],
  })
);

// Fallback for offline navigation
setCatchHandler(async ({ request }) => {
  // Return offline page for document requests
  if (request.destination === 'document') {
    const cache = await caches.open('precache');
    const offlineResponse = await cache.match('/offline');
    if (offlineResponse) return offlineResponse;
  }
  
  // Return placeholder for images
  if (request.destination === 'image') {
    return new Response(
      '<svg width="400" height="300" xmlns="http://www.w3.org/2000/svg"><rect width="100%" height="100%" fill="#f0f0f0"/><text x="50%" y="50%" dominant-baseline="middle" text-anchor="middle" fill="#999">Offline</text></svg>',
      { headers: { 'Content-Type': 'image/svg+xml' } }
    );
  }
  
  return Response.error();
});

// Background sync for form submissions
self.addEventListener('sync', (event) => {
  if (event.tag === 'sync-forms') {
    event.waitUntil(syncFormSubmissions());
  }
});

async function syncFormSubmissions() {
  const db = await openDB('form-queue', 1);
  const submissions = await db.getAll('submissions');
  
  for (const submission of submissions) {
    try {
      const response = await fetch(submission.url, {
        method: submission.method,
        headers: submission.headers,
        body: submission.body,
      });
      
      if (response.ok) {
        await db.delete('submissions', submission.id);
        // Notify clients of successful sync
        const clients = await self.clients.matchAll();
        clients.forEach(client => {
          client.postMessage({
            type: 'SYNC_COMPLETE',
            id: submission.id,
          });
        });
      }
    } catch (error) {
      console.error('Background sync failed:', error);
    }
  }
}

// Handle push notifications
self.addEventListener('push', (event) => {
  const data = event.data?.json() ?? {};
  
  const options = {
    body: data.body || 'New notification',
    icon: '/icons/icon-192x192.png',
    badge: '/icons/badge-72x72.png',
    tag: data.tag || 'default',
    requireInteraction: false,
    data: data.url || '/',
    actions: data.actions || [],
    vibrate: [200, 100, 200],
  };

  event.waitUntil(
    self.registration.showNotification(data.title || 'Update', options)
  );
});

// Handle notification clicks
self.addEventListener('notificationclick', (event) => {
  event.notification.close();
  
  event.waitUntil(
    clients.openWindow(event.notification.data)
  );
});
```

## 29.3 Offline Fallback Pages

Create graceful degradation experiences when users lose connectivity.

### Network Status Detection and Offline UI

```typescript
// hooks/use-network-status.ts
'use client';

import { useState, useEffect } from 'react';

export function useNetworkStatus() {
  const [isOnline, setIsOnline] = useState(true);
  const [connectionType, setConnectionType] = useState<string>('unknown');
  const [saveData, setSaveData] = useState(false);

  useEffect(() => {
    const handleOnline = () => setIsOnline(true);
    const handleOffline = () => setIsOnline(false);
    
    const updateConnectionStatus = () => {
      if ('connection' in navigator) {
        const conn = (navigator as any).connection;
        setConnectionType(conn.effectiveType || 'unknown');
        setSaveData(conn.saveData || false);
      }
    };

    window.addEventListener('online', handleOnline);
    window.addEventListener('offline', handleOffline);
    
    if ('connection' in navigator) {
      (navigator as any).connection.addEventListener('change', updateConnectionStatus);
      updateConnectionStatus();
    }

    // Initial check
    setIsOnline(navigator.onLine);

    return () => {
      window.removeEventListener('online', handleOnline);
      window.removeEventListener('offline', handleOffline);
      if ('connection' in navigator) {
        (navigator as any).connection.removeEventListener('change', updateConnectionStatus);
      }
    };
  }, []);

  return { isOnline, connectionType, saveData };
}

// components/offline-banner.tsx
'use client';

import { useNetworkStatus } from '@/hooks/use-network-status';
import { useEffect, useState } from 'react';

export function OfflineBanner() {
  const { isOnline, connectionType } = useNetworkStatus();
  const [showBanner, setShowBanner] = useState(false);

  useEffect(() => {
    if (!isOnline) {
      setShowBanner(true);
    } else {
      // Keep banner visible briefly when coming back online
      const timer = setTimeout(() => setShowBanner(false), 3000);
      return () => clearTimeout(timer);
    }
  }, [isOnline]);

  if (!showBanner) return null;

  return (
    <div className={`fixed top-0 left-0 right-0 z-50 p-3 text-center transition-colors ${
      isOnline ? 'bg-green-500' : 'bg-red-500'
    } text-white`}>
      <div className="flex items-center justify-center gap-2">
        {isOnline ? (
          <>
            <WifiIcon className="w-5 h-5" />
            <span>Back online{connectionType !== 'unknown' && ` (${connectionType})`}</span>
          </>
        ) : (
          <>
            <WifiOffIcon className="w-5 h-5" />
            <span>You are offline. Some features may be unavailable.</span>
          </>
        )}
      </div>
    </div>
  );
}
```

### Offline Fallback Page

```typescript
// app/offline/page.tsx
export const dynamic = 'force-dynamic';

export default function OfflinePage() {
  return (
    <div className="min-h-screen flex items-center justify-center bg-gray-50 p-4">
      <div className="max-w-md w-full text-center">
        <div className="mb-6">
          <div className="w-24 h-24 bg-gray-200 rounded-full flex items-center justify-center mx-auto mb-4">
            <WifiOffIcon className="w-12 h-12 text-gray-400" />
          </div>
          <h1 className="text-3xl font-bold text-gray-900 mb-2">You're Offline</h1>
          <p className="text-gray-600 mb-6">
            Don't worry! You can still access previously loaded content. We'll sync your changes when you're back online.
          </p>
        </div>

        <div className="space-y-3">
          <button
            onClick={() => window.location.reload()}
            className="w-full px-4 py-3 bg-blue-600 text-white rounded-lg hover:bg-blue-700 transition-colors"
          >
            Try Again
          </button>
          
          <CachedContentList />
        </div>

        <div className="mt-8 p-4 bg-yellow-50 border border-yellow-200 rounded-lg">
          <p className="text-sm text-yellow-800">
            <strong>Tip:</strong> Install this app to your home screen for better offline access.
          </p>
        </div>
      </div>
    </div>
  );
}

// components/cached-content-list.tsx
'use client';

import { useEffect, useState } from 'react';
import Link from 'next/link';

interface CachedPage {
  url: string;
  title: string;
  timestamp: number;
}

export function CachedContentList() {
  const [cachedPages, setCachedPages] = useState<CachedPage[]>([]);

  useEffect(() => {
    // Query the cache API for available pages
    async function getCachedPages() {
      const cacheNames = await caches.keys();
      const pages: CachedPage[] = [];
      
      for (const cacheName of cacheNames) {
        const cache = await caches.open(cacheName);
        const requests = await cache.keys();
        
        for (const request of requests) {
          const url = new URL(request.url);
          // Only show HTML pages, not API calls or assets
          if (url.pathname.startsWith('/') && !url.pathname.startsWith('/api/')) {
            const response = await cache.match(request);
            if (response) {
              // Try to extract title from cached HTML
              const text = await response.clone().text();
              const titleMatch = text.match(/<title>(.*?)<\/title>/);
              const title = titleMatch ? titleMatch[1] : url.pathname;
              
              pages.push({
                url: url.pathname,
                title,
                timestamp: Date.now(), // Ideally extract from response headers
              });
            }
          }
        }
      }
      
      // Remove duplicates
      const unique = Array.from(new Map(pages.map(p => [p.url, p])).values());
      setCachedPages(unique.slice(0, 5)); // Show top 5
    }

    getCachedPages();
  }, []);

  if (cachedPages.length === 0) return null;

  return (
    <div className="text-left">
      <h3 className="text-sm font-medium text-gray-700 mb-3">Available Offline:</h3>
      <div className="space-y-2">
        {cachedPages.map((page) => (
          <Link
            key={page.url}
            href={page.url}
            className="block p-3 bg-white border rounded-lg hover:bg-gray-50 transition-colors"
          >
            <div className="font-medium text-gray-900">{page.title}</div>
            <div className="text-sm text-gray-500">{page.url}</div>
          </Link>
        ))}
      </div>
    </div>
  );
}
```

## 29.4 Background Sync

Implement deferred actions that execute when connectivity returns.

### Form Queue with Background Sync

```typescript
// lib/offline-queue.ts
import { openDB } from 'idb';

interface QueuedRequest {
  id: string;
  url: string;
  method: string;
  headers: Record<string, string>;
  body: any;
  timestamp: number;
  retries: number;
}

const DB_NAME = 'offline-queue';
const STORE_NAME = 'requests';

export async function queueRequest(request: Omit<QueuedRequest, 'id' | 'timestamp' | 'retries'>) {
  const db = await openDB(DB_NAME, 1, {
    upgrade(db) {
      if (!db.objectStoreNames.contains(STORE_NAME)) {
        db.createObjectStore(STORE_NAME, { keyPath: 'id' });
      }
    },
  });

  const queuedItem: QueuedRequest = {
    ...request,
    id: crypto.randomUUID(),
    timestamp: Date.now(),
    retries: 0,
  };

  await db.add(STORE_NAME, queuedItem);
  
  // Register for background sync if supported
  if ('serviceWorker' in navigator && 'SyncManager' in window) {
    const registration = await navigator.serviceWorker.ready;
    try {
      await registration.sync.register('sync-forms');
    } catch (error) {
      console.error('Background sync registration failed:', error);
    }
  }

  return queuedItem.id;
}

export async function processQueue() {
  const db = await openDB(DB_NAME, 1);
  const requests = await db.getAll(STORE_NAME);
  
  const results = await Promise.allSettled(
    requests.map(async (req) => {
      try {
        const response = await fetch(req.url, {
          method: req.method,
          headers: req.headers,
          body: JSON.stringify(req.body),
        });
        
        if (response.ok) {
          await db.delete(STORE_NAME, req.id);
          return { success: true, id: req.id };
        } else {
          throw new Error(`HTTP ${response.status}`);
        }
      } catch (error) {
        // Increment retry count
        await db.put(STORE_NAME, { ...req, retries: req.retries + 1 });
        throw error;
      }
    })
  );
  
  return results;
}

// hooks/use-offline-form.ts
'use client';

import { useState, useCallback } from 'react';
import { useNetworkStatus } from './use-network-status';
import { queueRequest } from '@/lib/offline-queue';

export function useOfflineForm<T>(options: {
  url: string;
  method?: string;
  onSuccess?: (data: T) => void;
  onError?: (error: Error) => void;
}) {
  const { isOnline } = useNetworkStatus();
  const [isSubmitting, setIsSubmitting] = useState(false);
  const [pendingId, setPendingId] = useState<string | null>(null);

  const submit = useCallback(async (data: any) => {
    setIsSubmitting(true);
    
    try {
      if (isOnline) {
        // Submit directly if online
        const response = await fetch(options.url, {
          method: options.method || 'POST',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify(data),
        });
        
        if (!response.ok) throw new Error('Submission failed');
        
        const result = await response.json();
        options.onSuccess?.(result);
      } else {
        // Queue for background sync if offline
        const id = await queueRequest({
          url: options.url,
          method: options.method || 'POST',
          headers: { 'Content-Type': 'application/json' },
          body: data,
        });
        
        setPendingId(id);
        // Show user that it will be synced later
        return { queued: true, id };
      }
    } catch (error) {
      options.onError?.(error as Error);
      throw error;
    } finally {
      setIsSubmitting(false);
    }
  }, [isOnline, options]);

  return {
    submit,
    isSubmitting,
    pendingId,
    isQueued: !!pendingId,
  };
}

// Usage in component
// components/contact-form.tsx
'use client';

import { useOfflineForm } from '@/hooks/use-offline-form';

export function ContactForm() {
  const { submit, isSubmitting, isQueued } = useOfflineForm({
    url: '/api/contact',
    onSuccess: () => alert('Message sent!'),
    onError: (err) => alert('Error: ' + err.message),
  });

  const handleSubmit = async (e: React.FormEvent<HTMLFormElement>) => {
    e.preventDefault();
    const formData = new FormData(e.currentTarget);
    await submit(Object.fromEntries(formData));
  };

  return (
    <form onSubmit={handleSubmit} className="space-y-4">
      {isQueued && (
        <div className="p-4 bg-yellow-50 text-yellow-800 rounded-lg">
          You're offline. Your message will be sent when you reconnect.
        </div>
      )}
      
      <div>
        <label className="block text-sm font-medium mb-1">Name</label>
        <input name="name" required className="w-full border rounded-lg p-2" />
      </div>
      
      <div>
        <label className="block text-sm font-medium mb-1">Message</label>
        <textarea name="message" required className="w-full border rounded-lg p-2" rows={4} />
      </div>
      
      <button
        type="submit"
        disabled={isSubmitting}
        className="w-full bg-blue-600 text-white py-2 rounded-lg disabled:opacity-50"
      >
        {isSubmitting ? 'Sending...' : isQueued ? 'Queued for Sync' : 'Send Message'}
      </button>
    </form>
  );
}
```

## 29.5 Installation Prompts and A2HS

Control the "Add to Home Screen" experience and track installation metrics.

### Custom Install Prompt

```typescript
// hooks/use-install-prompt.ts
'use client';

import { useState, useEffect, useCallback } from 'react';

interface BeforeInstallPromptEvent extends Event {
  prompt: () => Promise<void>;
  userChoice: Promise<{ outcome: 'accepted' | 'dismissed' }>;
}

export function useInstallPrompt() {
  const [prompt, setPrompt] = useState<BeforeInstallPromptEvent | null>(null);
  const [isInstalled, setIsInstalled] = useState(false);
  const [isStandalone, setIsStandalone] = useState(false);

  useEffect(() => {
    // Check if already installed (standalone mode)
    if (window.matchMedia('(display-mode: standalone)').matches) {
      setIsInstalled(true);
      setIsStandalone(true);
    }

    const handleBeforeInstallPrompt = (e: Event) => {
      e.preventDefault();
      setPrompt(e as BeforeInstallPromptEvent);
    };

    const handleAppInstalled = () => {
      setIsInstalled(true);
      setPrompt(null);
      // Track installation
      if (typeof window !== 'undefined' && (window as any).gtag) {
        (window as any).gtag('event', 'pwa_install', {
          event_category: 'engagement',
          event_label: 'Install Prompt Accepted',
        });
      }
    };

    window.addEventListener('beforeinstallprompt', handleBeforeInstallPrompt);
    window.addEventListener('appinstalled', handleAppInstalled);

    return () => {
      window.removeEventListener('beforeinstallprompt', handleBeforeInstallPrompt);
      window.removeEventListener('appinstalled', handleAppInstalled);
    };
  }, []);

  const showInstallPrompt = useCallback(async () => {
    if (!prompt) return { outcome: 'dismissed' as const };
    
    await prompt.prompt();
    const result = await prompt.userChoice;
    
    if (result.outcome === 'accepted') {
      setPrompt(null);
    }
    
    return result;
  }, [prompt]);

  return {
    isInstallable: !!prompt,
    isInstalled,
    isStandalone,
    showInstallPrompt,
  };
}

// components/install-button.tsx
'use client';

import { useInstallPrompt } from '@/hooks/use-install-prompt';
import { useState } from 'react';

export function InstallButton() {
  const { isInstallable, isInstalled, showInstallPrompt } = useInstallPrompt();
  const [showIOSInstructions, setShowIOSInstructions] = useState(false);

  // Detect iOS (which doesn't support beforeinstallprompt)
  const isIOS = /iPad|iPhone|iPod/.test(navigator.userAgent);

  if (isInstalled) return null;

  if (isIOS) {
    return (
      <>
        <button
          onClick={() => setShowIOSInstructions(true)}
          className="px-4 py-2 bg-gray-800 text-white rounded-lg text-sm"
        >
          Install App
        </button>
        
        {showIOSInstructions && (
          <div className="fixed inset-0 bg-black/50 flex items-end sm:items-center justify-center p-4 z-50">
            <div className="bg-white rounded-lg p-6 max-w-sm w-full">
              <h3 className="font-bold text-lg mb-2">Install to Home Screen</h3>
              <p className="text-gray-600 mb-4">
                Tap the share button in your browser, then select "Add to Home Screen"
              </p>
              <button
                onClick={() => setShowIOSInstructions(false)}
                className="w-full py-2 bg-gray-100 rounded-lg"
              >
                Got it
              </button>
            </div>
          </div>
        )}
      </>
    );
  }

  if (!isInstallable) return null;

  return (
    <button
      onClick={showInstallPrompt}
      className="px-4 py-2 bg-blue-600 text-white rounded-lg text-sm flex items-center gap-2"
    >
      <DownloadIcon className="w-4 h-4" />
      Install App
    </button>
  );
}
```

## Key Takeaways from Chapter 29

1. **next-pwa Configuration**: Use `next-pwa` to generate service workers with sensible defaults, but customize `runtimeCaching` for your specific needs. Use `NetworkFirst` for API calls, `CacheFirst` for static assets, and `StaleWhileRevalidate` for dynamic content.

2. **Service Worker Strategies**: Implement precaching for critical static assets, runtime caching for dynamic content, and custom fallbacks for offline scenarios. Use `BackgroundSyncPlugin` to queue failed requests automatically.

3. **Offline UX**: Always provide visual feedback for network status changes using `navigator.onLine` and the `online/offline` events. Create dedicated offline fallback pages that show cached content and explain limitations.

4. **Background Sync**: Queue user actions (forms, likes, comments) when offline using IndexedDB and the Background Sync API. Process the queue when connectivity returns, providing user feedback on sync status.

5. **Install Experience**: Handle the `beforeinstallprompt` event to customize the Add to Home Screen experience. Detect standalone mode to hide install buttons for already-installed users, and provide iOS-specific instructions since Safari doesn't support the standard prompt.

## Coming Up Next

**Chapter 30: AI Integration**

With your application now installable and resilient to network failures, it's time to add intelligent capabilities. In Chapter 30, we'll explore integrating AI features using the Vercel AI SDK, implementing streaming chat interfaces, building AI-powered content generation, and optimizing AI costs while maintaining privacy. You'll learn how to seamlessly blend traditional Next.js patterns with modern AI capabilities to create smarter, more responsive applications.